# Escalonando para GPU Nvidia com C++ std. Library

# Introdução

Como sabemos, temos instruções padroes de C++ que permitem informar que temos tipos diferentes de execução


## Uso de Lambdas em C++

Simplificação da criação de objetos, por exemplo: 

```cpp

std::vector<double> v ={1,2,3,4};
double s = 2.;

auto f=[s,&v](int idx) {return v[idx]*s;};
assert(f(1)==4);

```

Neste lambda, temos como parametros da função f (que está construida como um objeto) o indice, vetor e s, implicitamente. Essas variaveis chamamos de “lambda capture”.

- [s] captura por “valor”, fazendo uma cópia
- [&v] captura por “referencia”, armazena ponteiro.
- [=, &] ou [=,]  ⇒ podemos escolher todos de uma vez.

Lambda Captures permite o uso de atribuir variaveis que estao dentro do lambda

```cpp
auto f=[a = s, x = v.data()](int idx) {return x[idx]*a;};
```

 Por baixo dos panos, o compilador faz a seguinte estrutura:

```cpp
struct __unnamed {
	double s;
	std::vector<double> &v;
	double operator()(int idx){ return v[idx] *s;}
}; 
__unnamed f{s,v}; 
assert(f(1) == 4);
```


## Usando algoritmos de STL

podemos fazer uso do algoritmo `transform` de c++. Esta ferramenta permite o uso de transformações nos elementos um a um, por exemplo:

```cpp
std::vector<double> v = {1,2,3,4}, w(4);
std::transform(begin(v), end(v), begin(w), [](const double& e1){
	return 2.* e1;
});
```
Definimos o range de elementos(begin(v) até end(v)) e depois definimos o começo da saída (begin(w)). E mais, podemos atribuir diretivas sobre o método de execução destes elementos no método de transformação: 

```cpp
std::vector<double> v = {1,2,3,4}, w(4);
std::transform(std::execution::par, begin(v), end(v), begin(w), [](const double& e1){
	return 2.* e1;
});
```



## Observações 

1. quando fazemos o uso de `std::execution::par`, devemos especificar a arquitetura que desejamos utilizar para execução por meio de flags no processo de construção do arquivo.

|Plataforma|diretiva|
|------|-------|
|CPU|nvc++ -stdpar=multicore |
|GPU|nvc++ -stdpar=gpu |

2. Dados em stack atrapalham a execução do algoritmo em questão. Isso se deve pela questão da transferencia dos dados. Se enviamos via lambda um endereço `[&]`, isso é problematico para o envio de dados para GPU. Uma boa prática seria usar `[=]`
```cpp
//Mude disto
void multiply_with(vector<double>&v, double a){
	std::for_each(std::execution::par, 
		begin(v), end(v),
		[&](double& x) { x*=a; }
	);
}

//Para isso
void multiply_with(vector<double>&v, double a){
	std::for_each(std::execution::par, 
		begin(v), end(v),
		[=](double& x) { x*=a; }
	);
}
```
## Política de Execução
|Politica de Exec.|Onde ocorre a operação|Tipo de Operação|implementação assume que|
|-|-|-|-|
|`std::execution::seq`      |na thread chamada|Totalmente sequencial| Nada, execução sequencial|
|`std::execution::unseq`    |na thread chamada|Sem sequencia|Operação Independente - SIMD|
|`std::execution::par`      |multiplas threads|indeterminadamente sequencial entre cada thread|seguro paralelizar com multiplas threads|
|`std::execution::par_unseq`|multiplas threads|Sem sequencia| par+ unseq |

### Exemplos de Aplicação

In [132]:
%pycat std_lib/par_01.cpp

#include <iostream> 
#include <execution> 
#include <vector>
#include <algorithm>
#include <chrono>
#include <random>
#include <execution>

namespace ex = std::execution;


void foo(std::vector<int>& v, auto pol) {
    std::for_each(pol, v.begin(), v.end(), [=](int& x) {
        x *= 2;
    });
    
}
void using_lambda(auto pol, std::vector<int> &input, std::vector<int> x, int a){
    // Declarando funcao lambda
    auto f = [x = x.data(), size = x.size()](int i) {
        int acc = 0;
        for (int j = 0; j < size; j++) {
            acc += j * x[j];
        }
        return acc * i;
    };


    std::for_each(pol,          //tipo de execução
    begin(input), end(input),   //inicio-entrada, final-entrada 
    [&] (int &el){              //valor, elemento de entrada-saida
        el = f(el);             //funcao-lamda
    });    
}

enum exec {seq, par, par_unseq};

int main(int argc, char **argv){
    if(argc > 3 || argc ==1){
        std::cout << "Invalid input\n";
        return

In [152]:
# !nvcc -std=c++20 -O4 -fast -tp=native -Mllvm-fast -DNDEBUG -Mllvm-fast -o std_lib/par_01 std_lib/par_01.cpp
!g++ -std=c++20 -o std_lib/par_01 std_lib/par_01.cpp -ltbb
!time ./std_lib/par_01 0 100000000
!time ./std_lib/par_01 1 100000000
!time ./std_lib/par_01 2 100000000

Execucao do tipo: seq

real	0m2,764s
user	0m2,675s
sys	0m0,089s
Execucao do tipo: par

real	0m2,000s
user	0m4,329s
sys	0m0,093s
Execucao do tipo: par_unseq

real	0m1,956s
user	0m3,495s
sys	0m0,105s



#### _Observações:_

a diretiva de execução `par` e `par_unseq` têm comportamento parecido, mas é necessário atenção com relação a condições de corrida entre threads. Vejamos um exemplo


In [153]:
%pycat std_lib/par_02.cpp


#include <iostream> 
#include <execution> 
#include <vector>
#include <algorithm>
#include <chrono>
#include <random>
namespace execution = std::execution; 


int data_race(auto policy, std::vector<int> v){

    int sum = 0.;
    std::for_each(policy, v.begin(), v.end(), [&](int i){
        sum+= i;
    });
    //# std::sort(policy, v.begin(), v.end());
    return sum;    
}


int main(){
    int size = 10000;
    std::vector<int> v(size);  
    for(int i=0; i < size; i++){
        v[i] = rand() %1000;
    }
    
    std::cout <<"seq:      "<< data_race(execution::seq, v)<< std::endl;
    std::cout <<"unseq:    "<< data_race(execution::unseq, v)<< std::endl;
    std::cout <<"par_unseq:"<< data_race(execution::par_unseq, v)<< std::endl;
    std::cout <<"par:      "<< data_race(execution::par, v)<< std::endl;
    
    

}


In [66]:
!g++ -std=c++23 -o std_lib/par_01 std_lib/par_01.cpp -ltbb && ./std_lib/par_01

seq:      4969211
unseq:    4969211
par_unseq:4738498
par:      1203624


In [196]:
%%writefile std_lib/par_02_correct.cpp
#include <iostream> 
#include <execution> 
#include <vector>
#include <algorithm>
#include <chrono>
#include <random>
#include <mutex>
#include <numeric>
#include <functional>
namespace execution = std::execution; 


int ParSeq(std::vector<int>& v){
    int sum = 0;
    std::mutex m;

    std::for_each(execution::par, v.begin(), v.end(), [&](int i){
        std::lock_guard<std::mutex> lock(m);
        sum += i;
    });
     
    return sum;    
}

int ParUnseq_v1(std::vector<int> v){
    std::atomic<int> sum{0};
    std::for_each(execution::par_unseq, v.begin(), v.end(), 
    [&](int i){ sum+=i; });
    
    return sum.load();    
}
int ParUnseq_v2(std::vector<int> v){
    std::vector<int> w(v.size());
    int res = std::reduce(execution::par_unseq, v.begin(), v.end(),
    0, std::plus<int>{});
    return res;
}

int main(){
    int size = 10000;
    std::vector<int> v(size);  
    for(int i=0; i < size; i++){
        v[i] = rand() %1000;
    }
    
    std::cout <<"ParSeq     : "<< ParSeq(v)<< std::endl;
    std::cout <<"ParUnseq_v1: "<< ParUnseq_v1(v)<< std::endl;
    std::cout <<"ParUnseq_v2: "<< ParUnseq_v2(v)<< std::endl;
    
    

}

Overwriting std_lib/par_02_correct.cpp


In [197]:
# !nvcc -std=c++23 -o std_lib/par_02 std_lib/par_02.cpp -ltbb && ./std_lib/par_02
!g++ -std=c++23 -o std_lib/par_02_correct std_lib/par_02_correct.cpp -ltbb && ./std_lib/par_02_correct


ParSeq     : 4969211
ParUnseq_v1: 4969211
ParUnseq_v2: 4969211


## CheatSheet - STL 
```cpp
// Iteration & Transform
    std::for_each, std::for_each_n
    std::transform, std::transform_reduce
    std::transform_inclusive_scan, std::transform_exclusive_scan

// Reduction
std::reduce, std::transform_reduce
std::is_sorted, std::is_sorted_until

//Searching 
std::find, std::find_if

//Reorder Elements
std::sort, std::stable_sort, std::partial_sort, std::partial_sort_copy
```


## Indexing, ranges e Views

Em C++, podemos usar o indice dos loops:
```cpp
vector<double> v = {1,2,3,4}; 
for(int i=0;i<4; ++i)
    v[i]=f(i);
```

Mas como realizar isso de maneira paralela? 

- __Opçao 1: Obter indice por meio do endereço - ineficiente__
```cpp
std::transform(begin(v), end(v), 
    [v=v.data()](const double &el){
        ptrdiff_i i = &el - v;
        return f(i);
    });
```
- __Opção 2: usar iterador  (mais frequente)__
```cpp 

thrust::counting_iterator<size_t> it{0};
assert(*it == 0);

std::for_each_n(it, v.size(),
    [v = v.data()] (size_t i){
        v[i] = f(i);
    });
```


- __Opção 3: usar c++20 Ranges e Views__
```cpp
//Iteracao 1D
auto ints = std::views::iota(0,4); 
std::for_each(std::execution::par, begin(ints), end(ints),
    [v = v.data()] (size_t i){
        v[i] = f(i);
    });

//Iteracao 2D 
namespace stdv = std::views; 
auto v = stdv::cartesian_product(stdv::iota(0,N), stdv::iota(0,M));

std::for_each(std::execution::par, begin(v), end(v), 
    [] (auto& e) {
        auto [i,j] = e;
    });
```




# Referencias Bibliográficas
- [GPU Acceleration with the C++ Standard Library](https://courses.nvidia.com/courses/course-v1:DLI+S-AC-08+V1/)
- [CPP Reference](https://en.cppreference.com/)